In [ ]:
import pickle

In [ ]:
import pandas as pd
import seaborn as sns
import re # Регулярные выражения.
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as sp
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.model_selection import validation_curve, StratifiedKFold

# Решение от которого отталкиваться - медианы по подкатегориям

In [ ]:
train_df.head()

In [ ]:
train_df1 = pd.read_csv('train_4_col.csv', delimiter='\t')
train_df1 = train_df1.loc[:,['id','price']]
train_df2 = pd.read_csv('train_4_col_2.csv', delimiter='\t')
train_df2 = train_df2.loc[:,['id','category','subcategory']]
train_df = train_df1.merge(train_df2, how='left', on='id')

In [ ]:
# медины по категориям, подкатегориям
meds_cat = train_df.groupby('category')['price'].median()
meds_subcat = train_df.groupby('subcategory')['price'].median()

In [ ]:
meds_subcat.to_frame().head()

In [ ]:
test_df = pd.read_csv('test_3_col_2.csv', delimiter='\t')
test_df = test_df.loc[:,['id','category','subcategory']]
test_df = test_df.merge(meds_subcat.to_frame(), how='left', on='subcategory')

In [ ]:
ms = meds_subcat.to_frame()

In [ ]:
# новая категория в тесте
test_df2 = pd.read_csv('test_3_col_2.csv', delimiter='\t')
test_df2[test_df2['subcategory']==2301]

In [ ]:
test_df3=pd.read_csv('test_3_col.csv', delimiter='\t')

In [ ]:
test_df3[test_df3['id']=='0fe6723933ab5606918dd9a5']

In [ ]:
train_df[train_df['category']==23]

In [ ]:
test_df.loc[test_df.price.isna(),'price'] = 0

In [ ]:
# медианы по подкатегориям (в копейках), нули в новой подкатегории
test_df.loc[:,['id','price']].to_csv('subm1.csv', index=False)

In [ ]:
# автомобили. заполним медианой 102
test_df.loc[test_df['price']==0,'price'] = 11500000

In [ ]:
test_df.loc[:,['id','price']].to_csv('subm2.csv', index=False)

In [ ]:
test_df['price'] = test_df['price']/100

In [ ]:
# нужны рубли
test_df.loc[:,['id','price']].to_csv('subm3.csv', index=False)

# Конвертация fields в json

In [ ]:
f = open('train_sample.pckl', "rb")
train_sample = pickle.load(f)

In [ ]:
train_fields = pd.read_csv('train_4_col_3.csv', delimiter='\t')

In [ ]:
train_sample.fields[2]

In [ ]:
js = json.loads(str)

In [ ]:
type(js)

In [ ]:
type(js[0])

In [ ]:
train_fields.loc[:,"fields_json"] = train_fields.fields.replace("None", "\"\"").replace("True","1").replace("False","0").replace("\'", "\"")

In [ ]:
train_fields.fields_json[100000]

In [ ]:
#сохранили fields в формате json
train_fields.to_csv("my_train_4_col_3.csv")

In [ ]:
train_fields.fields[0]

In [ ]:
train_sample.fields[0][2]['field']['field_type']

In [ ]:
train_sample.fields[110]['110']

In [ ]:
train_fields.fields[110]

In [ ]:
ax = sns.violinplot(x="category", y="price", data=train_df)

In [ ]:
ax = sns.violinplot(x=train_df["price"])

In [ ]:
train_df1 = pd.read_csv('train_4_col.csv', delimiter='\t')

In [ ]:
train_df2 = pd.read_csv('train_4_col_2.csv', delimiter='\t')
train_df = train_df1.merge(train_df2, how='left', on='id')

In [ ]:
train_df = train_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])

In [ ]:
train_df = pd.read_csv('new_text_pr_cats.csv')

In [ ]:
morph=pymorphy2.MorphAnalyzer()
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёA-Za-z]+\-[А-Яа-яЁёA-Za-z]+|[А-Яа-яЁёA-Za-z]+', str(text))
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
    return words
def getMeaningfullString(txt):
    res = ' '.join(getMeaningfullWords(txt))
    return res

In [ ]:
train_df.head()

In [ ]:
train_df['new_name'] = train_df['name'].apply(func = getMeaningfullString)
train_df['new_description'] = train_df['description'].apply(func = getMeaningfullString)

In [ ]:
train_df['new_description'] = train_df['description'].apply(func = getMeaningfullString)

In [ ]:
train_df = train_df.fillna('')

In [ ]:
def rlmse_score(y_test, y_hat):
    # Your code here
    n = len(y_test)
    rlmse = np.sqrt(1/n*np.sum(np.square(np.log(y_test+np.ones(n)) - np.log(y_hat + np.ones(n)))))
    return rlmse

# Эту функцию трогать не надо
def rlmse_scorer(estimator, X, y):
    y_hat = estimator.predict(X)
    
    return rlmse_score(y, y_hat)

In [ ]:

ohe = OneHotEncoder(categorical_features = [1])
X_subcat = ohe.fit_transform(train_df.loc[:,['category','subcategory']])


In [ ]:
X_subcat

In [ ]:
X_subcat = X_subcat.toarray()[:,0:23]

In [ ]:
X_subcat = X_subcat.toarray()[:,0:250]

In [ ]:
X_subcat.tofile("X_subcat_array.csv", sep="\t")

In [ ]:
y = train_df['price']/100

In [ ]:
y.to_csv("y.csv")

In [ ]:
counter_name = CountVectorizer()
#counter_descr = CountVectorizer()
X_name = counter_name.fit_transform(train_df['new_name'].values)
#X_descr = counter_descr.fit_transform(train_df['new_description'].values)

In [ ]:
X_subcat = sp.csr_matrix(X_subcat)

In [ ]:
X_subcat.shape

In [ ]:
#X = sp.hstack((X_subcat, X_name, X_descr))
X = sp.hstack((X_subcat, X_name))

In [ ]:
X_train, X_train_test, y_train, y_train_test = train_test_split(X_new, y_new, test_size=0.3, random_state=111)

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train,y_train)
y_hat = regressor.predict(X_train_test)
#out_of_bag_prediction_for_x = regressor.oob_prediction_

#print(rlmse_score(out_of_bag_prediction_for_x, x))

In [ ]:
print(rlmse_score(y_train_test, y_hat))

In [ ]:
X.shape

In [ ]:
regressor = ElasticNet(alpha=0.1, l1_ratio=10)
regressor.fit(X_train,y_train)

In [ ]:
y_hat = regressor.predict(X_train_test)
print(rlmse_score(y_train_test, y_hat))

In [ ]:
alpha_range = np.logspace(-3, 5, 20)

In [ ]:
l1_ratio = 0.5
10 - 6.198857525607565
1.0 - 6.1896718854981465
0.1 - 5.762364497363481
0.01 - 5.594739242709756

l1_ratio = 10
10 - 6.19244423861566
0.01 - 5.578056005862723
0.001 - 75.15656556569385

In [ ]:
alpha_range

In [ ]:
alpha_range = np.logspace(-3, 5, 20)
for alpha in alpha_range:
    regressor = ElasticNet(alpha=alpha, l1_ratio=0.5)
    regressor.fit(X_train,y_train)
    y_hat = regressor.predict(X_train_test)
    print(alpha, '\t', rlmse_score(y_train_test, y_hat))

In [ ]:
X.shape

# булевские переменные в 0-1.
Формируем фичи - 6 столбцов с булевскими переменными, 250 подкатегорий

In [ ]:
f = open('train_hack.pckl', "rb")
train_df = pickle.load(f)
feature_columns = ['can_buy','can_promote','contacts_visible','delivery_available','mortgage_available','payment_available']
train_df_features = train_df.loc[:,feature_columns]
train_df_features = train_df_features * 1
X = np.hstack((train_df_features,X_subcat))

In [ ]:
train_df_features.to_csv('train_df_features.csv')

In [ ]:
train_df_features = pd.read_csv('train_df_features.csv')

In [ ]:
train_df_features.head()

In [ ]:
X = np.hstack((train_df_features.loc[:,['delivery_available','payment_available']],X_subcat))

In [ ]:
train_df_features = train_df_features.drop(columns=['Unnamed: 0'])

In [ ]:
X.tofile("x.csv",sep="\t")

In [ ]:
X = np.fromfile("x.csv", sep="\t")
X = X.reshape((1748890, 256))
y = pd.read_csv("y.csv",header=None)
y=y.iloc[:,1]

In [ ]:
f = open('test_hack.pckl', "rb")
test_sample = pickle.load(f)

In [ ]:
X_test_subcat = ohe.transform(test_sample.loc[:,['category','subcategory']])
X_test_subcat = X_test_subcat.toarray()[:,0:250]
feature_columns = ['can_buy','can_promote','contacts_visible','delivery_available','mortgage_available','payment_available']
test_df_features = test_sample.loc[:,feature_columns]
test_df_features = test_df_features * 1
X_test = np.hstack((test_df_features,X_test_subcat))

In [ ]:
y_hat = regressor.predict(X_test)

In [ ]:
test_sample.shape

In [ ]:
test_sample.loc[:,'price'] = y_hat

In [ ]:
test_sample.loc[:,['id','price']].to_csv('subm4.csv', index=False)

In [ ]:
test_sample.loc[:,'price'] = y_hat * 100
test_sample.loc[:,['id','price']].to_csv('subm5.csv', index=False)

# аномалии как крайние значения
идея: мусором считать все, что не повторяется в других объявлениях (не успела)

In [ ]:
# уберем аномалии
X_new = X[train_df.groupby("subcategory").price.\
      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
y_new = y[train_df.groupby("subcategory").price.\
      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
regressor = ElasticNet(alpha=0.1, l1_ratio=10)
regressor.fit(X_new,y_new)
# rlmse на сплите = 2.568335764977625

In [ ]:
y_hat = regressor.predict(X_test)

In [ ]:
test_sample.loc[:,'price'] = y_hat
test_sample.loc[:,['id','price']].to_csv('subm6.csv', index=False)

In [ ]:
# посмотритм по субкатегориям частоты
cat_counts = train_df.groupby('subcategory')['price'].count()

In [ ]:
# среди feature_columns = ['can_buy','can_promote','contacts_visible','delivery_available','mortgage_available','payment_available']
# только 'delivery_available' и 'payment_available' имеют разные значения
feature_columns =['delivery_available', 'payment_available']
X.shape

In [ ]:
#subcat = train_df['subcategory']


# X_2 = np.delete(X, [3,5], 1)
# все равно плохо на такой выборке valid_scores = array([[6.21722723, 6.27396792, 6.09464087],
#       [6.26858403, 6.30624024, 6.06554924],
#       [6.26742859, 6.30514385, 6.063095  ],
#       [6.26735874, 6.30507743, 6.0629456 ]])
# попробуем убрать первые 4, оставить только субкатегорию
# блин! наоборот надо было. оставить 3 и 5


#X_2 = np.delete(X, [0,1,2,4],1)
#array([[5.67051601, 5.69405841, 5.69651048],
#       [6.12368325, 6.16309129, 5.92382398],
#       [6.2627221 , 6.30057416, 6.05835798],
#       [6.26513786, 6.3029242 , 6.06070606]])
#X_2 = np.delete(X, [0,1,2,3,4,5],1)
#array([[6.21722723, 6.27396792, 6.09464087],
#      [6.26858403, 6.30624024, 6.06554924],
#       [6.26742859, 6.30514385, 6.063095  ],
#       [6.26735874, 6.30507743, 6.0629456 ]])

# уберем аномалии
# X_2 = np.delete(X, [0,1,2,4],1)

# X_new = X_2[train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
# y_new = y[train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
X_2 = np.delete(X, [0,1,2,4],1)

#X_new = X_2[train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))).eq(1)]
#y_new = y[train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))).eq(1)]


In [ ]:
train_df['subcategory'] = train_df2['subcategory']

In [ ]:
#good = train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))).eq(1)
good = train_df.groupby("subcategory").price.\
      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)

In [ ]:
#X_tr.price = good

In [ ]:
#subcat = train_df['subcategory'][train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
#subcat = train_df['subcategory'][train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))).eq(1)]
#train_df.groupby("subcategory").price.\
#      transform(lambda x : (x<x.quantile(0.95))).eq(1)



In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=111)
alpha_range = [0.01, 0.1, 1, 2]
train_scores, valid_scores = validation_curve(regressor, X_new, y_new, param_name="alpha", param_range=alpha_range,
                                              cv=cv.split(X_new, subcat), scoring=rlmse_scorer)

In [ ]:
valid_scores

In [ ]:
X_tr = pd.concat([pd.DataFrame(X_2), good], axis=1)
X_train, X_train_test, y_train, y_train_test = train_test_split(X_tr, y, test_size=0.3, random_state=111)
y_train = y_train.loc[X_train.price == True]
X_train = X_train.loc[X_train.price == True]
X_train = X_train.iloc[:,0:252]
X_train_test = X_train_test.iloc[:,0:252]

In [ ]:
#regressor = ElasticNet(alpha=0.1, l1_ratio=10)
# 2.9643305151603365
regressor = LinearRegression()
# 1.7119132454915786
regressor.fit(X_train,y_train)
print(rlmse_scorer(regressor, X_train_test, y_train_test))

In [ ]:
#submit
y_train = y.loc[X_tr.price == True]
X_train = X_tr.loc[X_tr.price == True]


In [ ]:
X_train = X_train.iloc[:,0:252]
regressor = LinearRegression()
regressor.fit(X_train,y_train)

In [ ]:
f = open('test_hack.pckl', "rb")
test_sample = pickle.load(f)


In [ ]:
test_sample.loc[test_sample['subcategory'] == 2301, 'subcategory'] = 102
X_test_subcat = ohe.transform(test_sample.loc[:,['category','subcategory']])

In [ ]:
X_test_subcat.shape

In [ ]:
X_test_subcat = X_test_subcat.toarray()[:,0:250]
feature_columns = ['delivery_available','payment_available']
test_df_features = test_sample.loc[:,feature_columns]
test_df_features = test_df_features * 1
X_test = np.hstack((test_df_features,X_test_subcat))

In [ ]:
X_test_subcat.info

In [ ]:
y_hat = regressor.predict(X_test)

In [ ]:
test_sample.loc[:,'price'] = y_hat.round(decimals =2)
test_sample.loc[:,['id','price']].to_csv('subm7.csv', index=False)

In [ ]:
y1 = pd.read_csv('subm7.csv')

In [ ]:
y1.price = y1.price*100

In [ ]:
y1.to_csv('subm8.csv', index = False)

In [ ]:
test_sample.loc[test_sample.subcategory==102]

In [ ]:
X_train, X_train_test, y_train, y_train_test = train_test_split(X, y, test_size=0.3, random_state=111)

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
rlmse_scorer(regressor, X_train_test, y_train_test)

# добавляем фичи - существительные и латинские слова (бренды) из названия - первые наиболее часто встречаемые
# и биграммы из описания

In [ ]:
# добавляем название
# из описания наиболее часто встречаемые 3-граммы, 2-граммы, 1-граммы
morph=pymorphy2.MorphAnalyzer()
def getSimpleString(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёA-Za-z]+\-[А-Яа-яЁёA-Za-z]+|[А-Яа-яЁёA-Za-z]+', str(text))
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
            elif p.tag == pymorphy2.tagset.OpencorporaTag('LATN'):
                words.append(t)
                break
    return ' '.join(words)


In [ ]:
def getLatnNounString(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёA-Za-z]+\-[А-Яа-яЁёA-Za-z]+|[А-Яа-яЁёA-Za-z]+', str(text))
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['NOUN']:
                words.append(p.normal_form)
                break
            elif p.tag == pymorphy2.tagset.OpencorporaTag('LATN'):
                words.append(t)
                break
    return ' '.join(words)

In [ ]:
train_df = pd.read_csv('train_4_col.csv', delimiter='\t')

In [ ]:
train_df['new_name'] = train_df['name'].apply(func = getSimpleString)

In [ ]:
train_df['new_description'] = train_df['description'].apply(func = getSimpleString)

In [ ]:
train_df['new_name'].to_csv('new_name.csv')

In [ ]:
train_df['new_description'].to_csv('new_description.csv')

In [ ]:
new_descr = pd.read_csv('new_description.csv')
train_df['new_description'] = new_descr['Unnamed: 1']
train_df['new_description'] = train_df['new_description'].apply(lambda x: str(x).replace('век', ''))
train_df['new_description'] = train_df['new_description'].apply(lambda x: str(x).replace('секунда', ''))

In [ ]:
test_sample['new_description'] = test_sample['description'].apply(func = getSimpleString)
test_sample['new_description'] = test_sample['new_description'].apply(lambda x: str(x).replace('век', ''))
test_sample['new_description'] = test_sample['new_description'].apply(lambda x: str(x).replace('секунда', ''))

In [ ]:
test_sample['new_name_noun'] = test_sample['name'].apply(func = getLatnNounString)
test_sample['new_name_noun'] = test_sample['new_name_noun'].apply(lambda x: str(x).replace('век', ''))
test_sample['new_name_noun'] = test_sample['new_name_noun'].apply(lambda x: str(x).replace('секунда', ''))
X_test_descr2 = counter_descr2.transform(test_sample['new_description'].values)
X_test_name = counter_name.transform(test_sample['new_name_noun'].values)



In [ ]:
X_test_descr2 = sp.csr_matrix(X_test_descr2)
X_test_name = sp.csr_matrix(X_test_name)
X_test = sp.csr_matrix(X_test)



In [ ]:
X_test = sp.csr_matrix(X_test)

In [ ]:
t1 = test_df_features.astype('float64')

In [ ]:
t1 = sp.csr_matrix(t1)
t2 = sp.csr_matrix(X_test_subcat)

In [ ]:
test_df_features.values

In [ ]:
X_2_n_test = sp.hstack((t1, t2, X_test_name[:,sorted_sum_name_df.index[0:n_name]], X_test_descr2[:,sorted_sum2_df.index[0:n]]))

In [ ]:
y_hat = xg_reg.predict(X_2_n_test)
test_sample.loc[:,'price'] = y_hat.round(decimals =2)
test_sample.loc[:,['id','price']].to_csv('subm9.csv', index=False)
test_sample.loc[:,'price'] = test_sample.loc[:,'price']*100
test_sample.loc[:,['id','price']].to_csv('subm10.csv', index=False)



In [ ]:
counter_descr2 = CountVectorizer(ngram_range=(2,2))
X_descr2 = counter_descr2.fit_transform(train_df['new_description'].values)
X_descr2 = sp.csr_matrix(X_descr2)
sums = X_descr2.sum(axis=0)
ngrams2 = counter_descr2.get_feature_names()
sum2_df = pd.DataFrame({'ngrams2': ngrams2, 'sums': sums.tolist()[0]})
sorted_sum2_df = sum2_df.sort_values(by='sums',ascending=False)

In [ ]:
X = np.fromfile("x.csv", sep="\t")
X = X.reshape((1748890, 256))
y = pd.read_csv("y.csv",header=None)
y=y.iloc[:,1]
X = np.delete(X, [0,1,2,4], 1)
X = sp.csr_matrix(X)

In [ ]:
train_df['new_name_noun'] = train_df['name'].apply(func = getLatnNounString)
train_df['new_name_noun'] = train_df['new_name_noun'].apply(lambda x: str(x).replace('век', ''))
train_df['new_name_noun'] = train_df['new_name_noun'].apply(lambda x: str(x).replace('секунда', ''))
counter_name = CountVectorizer()
X_name = counter_name.fit_transform(train_df['new_name_noun'].values)
X_name = sp.csr_matrix(X_name)
sums_name = X_name.sum(axis=0)
ngrams_name = counter_name.get_feature_names()
sum_name_df = pd.DataFrame({'ngrams': ngrams_name, 'sums': sums_name.tolist()[0]})
sorted_sum_name_df = sum_name_df.sort_values(by='sums',ascending=False)

In [ ]:
sorted_sum_name_df.to_csv('sorted_sum_name_df.csv')
sorted_sum2_df.to_csv('sorted_sum2_df.csv')

In [ ]:
#2 булевских + субкатегории + n_name первых существительных из названия + n первых биграмм из описания
n = 500
n_name = 1000
#X_2_n = sp.hstack((X, X_name[:,sorted_sum_name_df.index[0:n_name]], X_descr2[:,sorted_sum2_df.index[0:n]]))
X_2_n = sp.hstack((X, X_name, X_descr2))

X_train, X_train_test, y_train, y_train_test = train_test_split(X_2_n, y, test_size=0.3, random_state=111)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
rlmse_scorer(regressor, X_train_test, y_train_test)

In [ ]:
# удаление строк в csr_matrix
def delete_rows_csr(mat, indices):
    """
    Remove the rows denoted by ``indices`` form the CSR sparse matrix ``mat``.
    """
    if not isinstance(mat, sp.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    indices = list(indices)
    mask = np.ones(mat.shape[0], dtype=bool)
    mask[indices] = False
    return mat[mask]

In [ ]:
indices = good.index[good == False]

In [ ]:
indices

In [ ]:
X_anom = delete_rows_csr(X_2_n.tocsr(), indices)

In [ ]:
X_anom.shape

In [ ]:
y_anom = y.loc[good == True]

In [ ]:
y_anom.shape

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
rlmse_scorer(regressor, X_train_test, y_train_test)

In [ ]:
#n = 2000 5.302126782142029
#n = 1000  5.302126782142029
#n=500 5.294529658685628
#n=200 5.33351401196549
#n=100 5.367065667856761

#n=500
#добавим n_name
#n_name = 1000 5.1358093668948

In [ ]:
import xgboost as xgb

In [ ]:
n = 500
n_name = 1000
X_2_n = sp.hstack((X, X_name[:,sorted_sum_name_df.index[0:n_name]], X_descr2[:,sorted_sum2_df.index[0:n]]))
X_train, X_train_test, y_train, y_train_test = train_test_split(X_2_n, y, test_size=0.3, random_state=111)




In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.005,
                max_depth = 10, alpha = 10, n_estimators = 10)


xg_reg.fit(X_2_n, y)

#rlmse_scorer(xg_reg, X_train_test, y_train_test)

In [ ]:
n = 5, n_name = 10
max_depth
5 5.66153325033493 

max_depth 10 learn 0.005 3.3807009004878967

n = 500 n_name = 10
3.3675370599640186

n = 500 n_name = 1000

In [ ]:
import matplotlib.pyplot as plt
data_dmatrix = xgb.DMatrix(data=X_2_n,label=y)

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
# посмотрим что без аномалий
train_df2 = pd.read_csv('train_4_col_2.csv', delimiter='\t')
train_df['subcategory'] = train_df2['subcategory']
good = train_df.groupby("subcategory").price.\
      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)

In [ ]:
X_tr = pd.concat([pd.DataFrame(X_2_n.toarray()), good], axis=1)
X_train, X_train_test, y_train, y_train_test = train_test_split(X_tr, y, test_size=0.3, random_state=111)
y_train = y_train.loc[X_train.price == True]
X_train = X_train.loc[X_train.price == True]

In [ ]:
X_train = X_train.iloc[:,0:1752]
X_train_test = X_train_test.iloc[:,0:1752]

In [ ]:
type(X_2_n)

In [ ]:
#subm 11 - без anomalies
xg_reg.fit(X_anom, y_anom)

In [ ]:
y_hat = xg_reg.predict(X_2_n_test)
test_sample.loc[:,'price'] = y_hat.round(decimals =2)
test_sample.loc[:,['id','price']].to_csv('subm11.csv', index=False)